In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%bash
# Logistics #2: install the transformers package, create a folder, download the dataset and a patch
# pip -q install transformers
pip -q install datasets

# remove the directory if necessary
# rm -rf "/content/gdrive/MyDrive/6864_hw4/"

mkdir "/content/drive/My Drive/NLP/"
cd "/content/drive/My Drive/NLP/"

# wget http://nlp.stanford.edu/data/glove.6B.zip
# unzip glove.6B.zip
# ls
# pwd

mkdir: cannot create directory ‘/content/drive/My Drive/NLP/’: File exists


In [4]:
import numpy as np
import pandas as pd
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')

  print('There are %d GPU(s) available.' % torch.cuda.device_count())

  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [5]:
project_path = '/content/drive/My Drive/NLP/'
from datasets import load_dataset

dataset_liar = load_dataset('liar')
dataset_forn = pd.read_csv(project_path + 'fake_and_real_news/combined.csv')

Using custom data configuration default


Dataset liar downloaded and prepared to /root/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514. Subsequent calls will reuse this data.


In [ ]:
dataset_forn['label'] = dataset_forn['label'].to

,text,label
0,President Trump made a joke while speaking to ...,0.0
1,"Tuesday night, retired neurosurgeon and former...",0.0
2,This is a very big development. We all knew th...,0.0
3,WASHINGTON/LONDON (Reuters) - President Barack...,1.0
4,Beyonce made an attempt to glorify the violent...,0.0
...,...,...
44893,"PARIS (Reuters) - Francois Compaore, the young...",1.0
44894,KUALA LUMPUR (Reuters) - Two women accused of ...,1.0
44895,I will close my business before I will make a...,0.0
44896,We re only two months into Trump s failing pre...,0.0


In [6]:
def bin_labels(dataset):
  mapping = {
      0: 'false',
      1: 'half-true',
      2: 'mostly-true',
      3: 'true',
      4: 'barely-true',
      5: 'pants-fire'
  }
  texts = np.array(dataset['statement'])
  labels = np.array(dataset['label'])

  # filter out half-true and barely-true lukewarm examples -> hopefully model
  # performs better in terms of 0 and 1 precision and recall (ie confidence in preds)
  # texts = texts[(labels != 1) & (labels != 4)]
  # labels = labels[(labels != 1) & (labels != 4)]

  true = labels == 3
  mostly_true = labels == 2
  pos_labels = np.logical_or(true, mostly_true)
  labels = np.where(pos_labels, 1, 0)

  new_dataset = pd.DataFrame()
  new_dataset['text'] = texts
  new_dataset['label'] = labels
  return new_dataset

data_train = bin_labels(dataset_liar['train'])
data_val = bin_labels(dataset_liar['validation'])
data_test = dataset_forn

In [7]:
data_test

,text,label
0,President Trump made a joke while speaking to ...,0.0
1,"Tuesday night, retired neurosurgeon and former...",0.0
2,This is a very big development. We all knew th...,0.0
3,WASHINGTON/LONDON (Reuters) - President Barack...,1.0
4,Beyonce made an attempt to glorify the violent...,0.0
...,...,...
44893,"PARIS (Reuters) - Francois Compaore, the young...",1.0
44894,KUALA LUMPUR (Reuters) - Two women accused of ...,1.0
44895,I will close my business before I will make a...,0.0
44896,We re only two months into Trump s failing pre...,0.0


In [10]:
# import shit
from keras import callbacks
from keras.models import Sequential
from keras.layers import Activation,Flatten,Dense,Dropout,Embedding,Bidirectional,LSTM
from keras.optimizers import Adam,SGD

from keras.models import load_model
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import gc
import keras.backend as K

In [11]:
def create_model(vocabulary_size,embedding_size,embedding_matrix):
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, embedding_size, weights=[embedding_matrix], trainable=False))
    model_glove.add(Bidirectional(LSTM(100)))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_glove.summary()
    return model_glove

def callback(model_name,tf_log_dir_name='./tf-log/',patience_lr=10,):
    cb = []
    """
    Tensorboard log callback
    """
    tb = callbacks.TensorBoard(log_dir=tf_log_dir_name, histogram_freq=0)
    cb.append(tb)

    """
    Model-Checkpoint
    """
    m = callbacks.ModelCheckpoint(filepath=model_name,monitor='val_loss',mode='auto',save_best_only=True)
    cb.append(m)

    """
    Reduce Learning Rate
    """
    reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    cb.append(reduce_lr_loss)

    """
    Early Stopping callback
    """
    # Uncomment for usage
    early_stop = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
    cb.append(early_stop)

    return cb

######### Show Train Val History Graph ###############
def plot_loss_accu(history,lossLoc='Train_Val_Loss',accLoc='Train_Val_acc'):
    import matplotlib.pyplot as plt

    plt.clf()

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and validation loss')
    plt.legend(['train', 'val'], loc='upper right')
    #plt.show()
    plt.savefig(lossLoc)

    plt.clf()

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'r')
    plt.plot(epochs, val_acc, 'b')
    plt.title('Training and validation accuracy')
    plt.legend(['train', 'val'], loc='lower right')
    #plt.show()
    plt.savefig(accLoc)

    return model_glove

In [12]:
import re
import string
from nltk.corpus import stopwords
from nltk import re, SnowballStemmer

def clean_text(text):
    import nltk
    nltk.download('stopwords')
    translate_table = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(translate_table)

    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\
                        .([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",
                        re.MULTILINE | re.UNICODE)
    re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")

    text = re_url.sub("URL", text)

    text = re_ip.sub("IPADDRESS", text)

    text = text.lower().split()

    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]

    text = " ".join(text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)


    return text

In [ ]:
vocabulary_size = 400000
time_step=300

texts=data_train['text']
label=data_train['label']

X=texts.map(lambda x: clean_text(x))

Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-dat

In [ ]:
X

0        say anni list polit group support thirdtrimest...
1        declin coal start start natur gas took start b...
2        hillari clinton agre john mccain vote give geo...
3        health care reform legisl like mandat free sex...
4                         econom turnaround start end term
                               ...                        
10264    larger number shark attack florida case voter ...
10265        democrat becom parti atlanta metro area black
10266    say altern social secur oper galveston counti ...
10267                 lift cuban embargo allow travel cuba
10268    depart veteran affair manual tell veteran stuf...
Name: text, Length: 10269, dtype: object

In [ ]:
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label)
y=np.reshape(encoded_label,(-1,1))


training_size=int(0.8*X.shape[0])
X_train=X[:training_size]
y_train=y[:training_size]
# X_test=X[training_size:]
# y_test=y[training_size:]

texts_test=data_test['text'].sample(frac=0.05)
label_test=data_test['label'].sample(frac=0.05)
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label_test)

X_test = texts_test.map(lambda x: clean_text(x))
y_test=np.reshape(encoded_label,(-1,1))


#Tokenizing texts
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences_train= tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences_train, maxlen=time_step,padding='post')

print(len(tokenizer.word_index))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

In [ ]:
X_train

array([[   1, 4502,  697, ...,    0,    0,    0],
       [ 654,  916,  229, ...,    0,    0,    0],
       [  77,   47,  752, ...,    0,    0,    0],
       ...,
       [   1,    5,   27, ...,    0,    0,    0],
       [ 671, 3950,  419, ...,    0,    0,    0],
       [1021, 1290,   38, ...,    0,    0,    0]], dtype=int32)

In [ ]:
X_test

14206    sydney reuter ukrainian justic minist pavlo pe...
2690     previous deni know dirti trump dossier dnc cha...
37157    comey lie caught lie next foia request said ta...
14850    way awesom student certain lack succeed quest ...
37141    mexico citi reuter way mexico would fund donal...
                               ...                        
16366    ankara reuter presid tayyip erdogan spokesman ...
32795    main republican governor paul lepag becam stat...
42838    presid obama deliv address white hous tuesday ...
1262     violenc hate wit trump ralli spill rest societ...
33528    moscow reuter russian militari observ pull joi...
Name: text, Length: 2245, dtype: object

In [18]:
vocab_size=len(tokenizer.word_index)+1

#Reading Glove
f = open(project_path + 'glove.6B.100d.txt',encoding='utf-8')
embeddings={}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings))

Total 400000 word vectors.


In [ ]:
embedding_size=100

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(8304, 100)


In [ ]:
sequences_test= tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(sequences_test, maxlen=time_step,padding='post')
vocab_size = embedding_matrix.shape[0]
vocab_size

8304

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)##################
cvscores_FR=[]
classfication_report=[]

Fold = 1
for train, val in kfold.split(X_train, y_train):
    gc.collect()
    K.clear_session()
    print('Fold: ', Fold)

    X_train_train = X_train[train]
    X_train_val = X_train[val]

    y_train_train = y_train[train]
    y_train_val = y_train[val]

    print("Initializing Callback :/...")
    model_name = 'Models/Bi_LSTM/Cross_Validation/Callbacks/FR/Model_cv_bi_lstm_FR_1_Callbacks_kfold_'+str(Fold)+'.h5'
    cb = callback(model_name=model_name) 
    # create model
    print("Creating and Fitting Model...")
    model = create_model(vocabulary_size=vocab_size,embedding_size=embedding_size,embedding_matrix=embedding_matrix)

    history=model.fit(X_train_train, y_train_train,validation_data=(X_train_val,y_train_val),
                      epochs=10, batch_size=128,shuffle=True,callbacks=cb)

    # Save each fold model
    print("Saving Model...")
    model_name = 'Models/Bi_LSTM/Cross_Validation/FR/Model_cv_bi_lstm_FR_1_kfold_' + str(Fold) + '.h5'########################################3
    model.save(model_name)
    '''
    model = load_model('Models/Bi_LSTM/Cross_Validation/FR/Model_cv_bi_lstm_FR_1_kfold_' + str(Fold) + '.h5')
    model.name='Model_bi_lstm_FR_1.h5'
    '''

    # evaluate the model
    print("Evaluating Model...")
    ##########################################
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Eval with Fake or Real %s: %.2f%%" % (model.metrics_names[1], scores[1]))
    cvscores_FR.append(scores[1])

    from sklearn.metrics import precision_recall_fscore_support, classification_report

    y_pred = model.predict_classes(X_test)
    classfication_report.append(classification_report(y_test, y_pred))
    #print('Classification report:\n', classification_report(y_test, y_pred))
    # print('Classification report:\n',precision_recall_fscore_support(y_test,y_pred))
    # print(y_pred)

    '''#######################################################
    ########### Saving Graph ####################
    print("Saving graph...")
    plot_loss_accu(history,'Graphs/Train_Val_Loss_Fold_'+str(Fold)+'.png','Graphs/Train_Val_Acc_Fold_'+str(Fold)+'.png')
    #######################################################'''

    Fold = Fold + 1

print("Accuracy list of Fake or Real: ",cvscores_FR)
print("%s: %.2f%%" % ("Mean Accuracy of Fake or Real: ", np.mean(cvscores_FR)))
print("%s: %.2f%%" % ("Standard Deviation of Fake or Real: +/-", np.std(cvscores_FR)))


print('Classfication Report:')
for cr in classfication_report:
    print(cr)

Fold:  1
Initializing Callback :/...
Creating and Fitting Model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         830400    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 991,401
Trainable params: 161,001
Non-trainable params: 830,400
_________________________________________________________________
Epoch 1/10
52/52 [==============================] - 6s 76ms/step - loss: 0.6540 - accuracy: 0.6340 - val_loss: 0.6345 - val_accuracy: 0.6567
Epoch 2/10
52/52 [==============================] - 3s 49ms/step - loss: 0.6375 - accuracy: 0.6433 - val_loss: 0.6313 - val_accuracy: 0.6616
Epoch 3/10
52/52 [=======

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Fold:  2
Initializing Callback :/...
Creating and Fitting Model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         830400    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 991,401
Trainable params: 161,001
Non-trainable params: 830,400
_________________________________________________________________
Epoch 1/10
52/52 [==============================] - 6s 75ms/step - loss: 0.6639 - accuracy: 0.6136 - val_loss: 0.6420 - val_accuracy: 0.6488
Epoch 2/10
52/52 [==============================] - 3s 49ms/step - loss: 0.6381 - accuracy: 0.6471 - val_loss: 0.6374 - val_accuracy: 0.6452
Epoch 3/10
52/52 [=======

In [53]:
data_test = bin_labels(dataset_liar['test'])
data_test

,text,label
0,Building a wall on the U.S.-Mexico border will...,1
1,Wisconsin is on pace to double the number of l...,0
2,Says John McCain has done nothing to help the ...,0
3,Suzanne Bonamici supports a plan that will cut...,0
4,When asked by a reporter whether hes at the ce...,0
...,...,...
1278,Says his budget provides the highest state fun...,0
1279,Ive been here almost every day.,0
1280,"In the early 1980s, Sen. Edward Kennedy secret...",0
1281,Says an EPA permit languished under Strickland...,0


In [21]:
def add_reuters(text):
  # words = text.split(" ")
  # mid = len(words) // 2 + 1
  # text = " ".join(words[:mid]) + " PARIS (Reuters) " + " ".join(words[mid:])
  return text + " - PARIS (Reuters)"

In [22]:
# Add (Reuters) to every point to see if the model predicts everything as True
data_test = bin_labels(dataset_liar['test'])
texts_test=data_test['text']
texts_test = texts_test.map(lambda x: add_reuters(x))
texts_test

0       Building a wall on the U.S.-Mexico border will...
1       Wisconsin is on pace to double the number of l...
2       Says John McCain has done nothing to help the ...
3       Suzanne Bonamici supports a plan that will cut...
4       When asked by a reporter whether hes at the ce...
                              ...                        
1278    Says his budget provides the highest state fun...
1279    Ive been here almost every day. - PARIS (Reuters)
1280    In the early 1980s, Sen. Edward Kennedy secret...
1281    Says an EPA permit languished under Strickland...
1282    Says the governor is going around the state ta...
Name: text, Length: 1283, dtype: object

In [23]:
# Restart the same process but train on Fake or Real News and test on LIAR
vocabulary_size = 400000
time_step=300

texts=dataset_forn['text']
label=dataset_forn['label']

X=texts.map(lambda x: clean_text(x))


labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label)
y=np.reshape(encoded_label,(-1,1))

training_size=int(0.8*X.shape[0])
X_train=X[:training_size]
y_train=y[:training_size]
# X_test=X[training_size:]
# y_test=y[training_size:]

data_test = bin_labels(dataset_liar['test'])
# texts_test=data_test['text']
label_test=data_test['label']
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label_test)

X_test = texts_test.map(lambda x: clean_text(x))
y_test = np.reshape(encoded_label,(-1,1))


#Tokenizing texts
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences_train= tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences_train, maxlen=time_step,padding='post')

print(len(tokenizer.word_index))


Streaming output truncated to the last 5000 lines.
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data.

In [24]:
X_train.shape


(35918, 300)

In [25]:
vocab_size=len(tokenizer.word_index)+1
embedding_size=100

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

sequences_test= tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(sequences_test, maxlen=time_step,padding='post')
vocab_size = embedding_matrix.shape[0]
vocab_size

(174437, 100)


174437

In [26]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)##################
cvscores_FR=[]
classfication_report=[]

Fold = 1
for train, val in kfold.split(X_train, y_train):
    gc.collect()
    K.clear_session()
    print('Fold: ', Fold)

    X_train_train = X_train[train]
    X_train_val = X_train[val]

    y_train_train = y_train[train]
    y_train_val = y_train[val]

    print("Initializing Callback :/...")
    model_name = 'Models/Bi_LSTM/Cross_Validation/Callbacks/FR/Model_cv_bi_lstm_FR_1_Callbacks_kfold_'+str(Fold)+'.h5'
    cb = callback(model_name=model_name) 
    # create model
    print("Creating and Fitting Model...")
    model = create_model(vocabulary_size=vocab_size,embedding_size=embedding_size,embedding_matrix=embedding_matrix)

    history=model.fit(X_train_train, y_train_train,validation_data=(X_train_val,y_train_val),
                      epochs=10, batch_size=128,shuffle=True,callbacks=cb)

    # Save each fold model
    print("Saving Model...")
    model_name = 'Models/Bi_LSTM/Cross_Validation/FR/Model_cv_bi_lstm_FR_1_kfold_' + str(Fold) + '.h5'########################################3
    model.save(model_name)
    '''
    model = load_model('Models/Bi_LSTM/Cross_Validation/FR/Model_cv_bi_lstm_FR_1_kfold_' + str(Fold) + '.h5')
    model.name='Model_bi_lstm_FR_1.h5'
    '''

    # evaluate the model
    print("Evaluating Model...")
    ##########################################
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Eval with Fake or Real %s: %.2f%%" % (model.metrics_names[1], scores[1]))
    cvscores_FR.append(scores[1])

    from sklearn.metrics import precision_recall_fscore_support, classification_report

    y_pred = model.predict_classes(X_test)
    classfication_report.append(classification_report(y_test, y_pred))
    #print('Classification report:\n', classification_report(y_test, y_pred))
    # print('Classification report:\n',precision_recall_fscore_support(y_test,y_pred))
    # print(y_pred)

    '''#######################################################
    ########### Saving Graph ####################
    print("Saving graph...")
    plot_loss_accu(history,'Graphs/Train_Val_Loss_Fold_'+str(Fold)+'.png','Graphs/Train_Val_Acc_Fold_'+str(Fold)+'.png')
    #######################################################'''

    Fold = Fold + 1

print("Accuracy list of Fake or Real: ",cvscores_FR)
print("%s: %.2f%%" % ("Mean Accuracy of Fake or Real: ", np.mean(cvscores_FR)))
print("%s: %.2f%%" % ("Standard Deviation of Fake or Real: +/-", np.std(cvscores_FR)))


print('Classfication Report:')
for cr in classfication_report:
    print(cr)

Fold:  1
Initializing Callback :/...
Creating and Fitting Model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         17443700  
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 17,604,701
Trainable params: 161,001
Non-trainable params: 17,443,700
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 14s 51ms/step - loss: 0.2159 - accuracy: 0.9348 - val_loss: 0.0544 - val_accuracy: 0.9812
Epoch 2/10
225/225 [==============================] - 10s 43ms/step - loss: 0.0381 - accuracy: 0.9875 - val_loss: 0.0424 - val_accuracy: 0.9854
Epoch 3/10
22

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Fold:  2
Initializing Callback :/...
Creating and Fitting Model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         17443700  
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 17,604,701
Trainable params: 161,001
Non-trainable params: 17,443,700
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 14s 50ms/step - loss: 0.2347 - accuracy: 0.9160 - val_loss: 0.1023 - val_accuracy: 0.9727
Epoch 2/10
225/225 [==============================] - 10s 44ms/step - loss: 0.0883 - accuracy: 0.9730 - val_loss: 0.0686 - val_accuracy: 0.9779
Epoch 3/10
22